In [154]:
import pandas as pd
import numpy as np
import os

In [155]:
path_homicidios = os.path.join('..','data','raw','homicidios.xlsx')

## ETL  Hechos

#### Extracción Hechos

In [156]:
hechos = pd.read_excel(path_homicidios,sheet_name='HECHOS')

In [157]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


### Transformación Hechos


**Datos duplicados**

Observamos que en este caso, no tenemos nungun dato duplciado.

In [158]:
hechos[hechos['ID'].duplicated()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


**Datos nulos**

A continuación analizaremos los datos nulos de la tabla hechos

In [159]:
pd.DataFrame(hechos.isna().sum(),columns=['Nulos'])

,Nulos
ID,0
N_VICTIMAS,0
FECHA,0
AAAA,0
MM,0
DD,0
HORA,0
HH,0
LUGAR_DEL_HECHO,0
TIPO_DE_CALLE,0


In [160]:
columns_with_nulls = ['Calle','Altura','Cruce','Dirección Normalizada']

- A continuación se observa que la columna altura y la columan Cruce contiene la mayor cantidad de nullos

- La columna Calle tiene un solo dato nulo.
- La columna Dirección Normalizada 8 nulos.
- La columna Cruce tiene $171$ nulos, siendo el 24,56% del los datos de la columna.
- La Altura tiene $567$ nulos que representan el 81,46% de datos de la columna


In [161]:
pd.DataFrame(hechos.isna().sum().loc[columns_with_nulls]/len(hechos),columns=['Porcentaje de nulos'])

,Porcentaje de nulos
Calle,0.001437
Altura,0.814655
Cruce,0.245690
Dirección Normalizada,0.011494


**Transformación de variables**
- Con la comunna altura y cruce podemos inferir que no pueden ocurrir ambas al mismo tiempo, por lo tanto si existe datos en una de ellas la otra deberá ser nula. Asi creamos una columna booleana, donde

$$\begin{cases}
1 && \text{Si el accidente fue en un cruce} \\
0 && \text{Si el accidente no fue en un cruce}\\
-1 && \text{No existe registro de la ubicación} 
\end{cases}$$

- Posteriormente vamos a remplazar los valores faltantes, con información de tipo de calle. por el momento dejaremos estos registros con -1

In [162]:
hechos['Cruce_boolean'] =  np.where(hechos['Cruce'].isnull() & hechos['Altura'].notnull(), 0,
        np.where(hechos['Cruce'].isnull() & hechos['Altura'].isnull(), -1,
        np.where(~hechos['Cruce'].isnull() & hechos['Altura'].isnull(), 1, np.nan)))

**Eliminación de columnas**

En esta sección eliminaremos las columnas que no utilizaremos en el proyecto.

- Dado que la dirección normalizada cuenta con la información del cruce o la altura respectivamente, procederemos a eliminarlas, ademas recordemos que contamos con la columna booleana que me informa si el accidente ocurrio en un cruce o no.

- La columna XY (CABA) nos podria servir para realizar una geolocalización de caba, pero se optó por buscar esta información en una base de datos oficial y posteriormente descargar un archivo geojson para realizar la vizualización del mapa de CABA, por lo tanto no necesitaremos esta columna para el analisis.  

In [163]:
hechos.drop(['Cruce','Altura', 'XY (CABA)'], axis=1,inplace=True)

**Tranformación de tipos** 

#### Llenado de nulos

- En el dataset se encuentran inconsistencias con la terminologia de los nullos, es por esto que se unificará con el string "SD" (sin datos). 

- En esta sección inspeccionaremos los datos para encontrar posibles nullos con este formato.

In [164]:
hechos['AAAA'].unique()
hechos['MM'].unique()
hechos['DD'].unique()
hechos['HORA'].unique()
hechos['HH'].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

- Anteriormente inspeccionamos si existia una fecha sin datos. Se observo que existe un dato sin hora (HORA) y sin franja horaria (HH). Este dato no es posible completarlo; dejaremos estos nulos ya que el objetivo es realizar un conteo de los usuarios que tuvieron un accidente fatal, es por esto que no eliminaremos registros. 

**Normalización de texto**

tomaremos las columnas con texto y las transformaremos en mayusculas.

In [165]:
hechos['LUGAR_DEL_HECHO'] = hechos['LUGAR_DEL_HECHO'].str.upper()
hechos['Calle'] = hechos['Calle'].str.upper()

Las calles que no tengan información las tomaremos como sin datos, ya que no tenemos información suficiente para inferirla

In [166]:
hechos.loc[hechos['Calle'].isna(),'Calle'] = 'SD'

La dirección normalizada es una concatenación de la información de la calle, avenida y cruce. por lo que a los datos nulos que tenemos en esta columna los llenaremos con la información del lugar del hecho.

In [167]:
hechos['Dirección Normalizada'] = hechos['Dirección Normalizada'].fillna(hechos['LUGAR_DEL_HECHO'])

In [168]:
hechos.isna().sum()

ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Dirección Normalizada    0
COMUNA                   0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
Cruce_boolean            0
dtype: int64

In [169]:
hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Dirección Normalizada',
       'COMUNA', 'pos x', 'pos y', 'PARTICIPANTES', 'VICTIMA', 'ACUSADO',
       'Cruce_boolean'],
      dtype='object')

**Transformación de columna ubicación**

En esta sección vamos a limpiar las columnas con la latitud y longitud del accidente. se quiere extraer la ubicación de los datos validos, es por esto que un dato sin información coherente lo tomaremos como nulo.

In [170]:
valid_positions =  ~(hechos['pos x'].str.contains('[^0-9.,-]') | hechos['pos y'].str.contains('[^0-9.,-]') | (hechos['pos x'] == '.') | (hechos['pos y'] == '.'))

hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

In [171]:
hechos.loc[~valid_positions] 

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Dirección Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,Cruce_boolean
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES PK 10000,13,NaN,NaN,MOTO-SD,MOTO,SD,-1.0
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,AU BUENOS AIRES - LA PLATA KM. 4,4,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,0,NaN,NaN,SD-SD,SD,SD,-1.0
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,9,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AU DELLEPIANE 2400,7,NaN,NaN,AUTO-AUTO,AUTO,AUTO,-1.0
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,NaN,NaN,MOTO-PASAJEROS,MOTO,PASAJEROS,-1.0
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES KM 4.7,14,NaN,NaN,PEATON-AUTO,PEATON,AUTO,-1.0
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV. KM 6,1",14,NaN,NaN,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0


**Renombrar columnas**

In [172]:
nombres_columnas = {
    'ID': 'id_hechos',
    'N_VICTIMAS': 'n_victimas',
    'FECHA': 'fecha',
    'AAAA': 'anho',
    'MM': 'mes',
    'DD': 'dia',
    'HORA': 'hora',
    'HH': 'franja_horaria',
    'LUGAR_DEL_HECHO': 'lugar_del_hecho',
    'TIPO_DE_CALLE': 'tipo_de_calle',
    'Calle': 'calle',
    'Dirección Normalizada': 'direccion_normalizada',
    'COMUNA': 'comuna',
    'pos x': 'longitud',
    'pos y': 'latitud',
    'PARTICIPANTES': 'participantes',
    'VICTIMA': 'victimas',
    'ACUSADO': 'acusado',
}

hechos.rename(columns=nombres_columnas, inplace=True)

**Analisis finales de hechos**

In [173]:
hechos[hechos['Cruce_boolean']==-1].sample(10)

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
346,2018-0072,1,2018-07-11,2018,7,11,13:15:00,13,"AU DELLEPIANE KILOMETRO 0.0, ALTURA CALLE MONT...",AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AUTOPISTA DELLEPIANE LUIS TTE. GRAL. y MONTIEL,8,-58.48267426,-34.68454456,MOTO-CARGAS,MOTO,CARGAS,-1.0
100,2016-0130,1,2016-10-04,2016,10,4,12:30:00,12,AUTOPISTA 9 DE JULIO SUR Y AV BRASIL,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y B...,1,-58.37910942,-34.62764717,MOTO-CARGAS,MOTO,CARGAS,-1.0
555,2020-0035,1,2020-08-04,2020,8,4,05:30:00,5,"AU 25 DE MAYO ALTURA PUMACAHUA Y RECUERO, CASI...",AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO y PUMACAHUA,7,-58.45068621,-34.63638927,MOTO-PASAJEROS,MOTO,PASAJEROS,-1.0
281,2018-0007,1,2018-01-11,2018,1,11,02:30:00,2,AU PERITO MORENO KM 0.6,AUTOPISTA,AUTOPISTA PERITO MORENO,AUTOPISTA PERITO MORENO KM. 0.6,9,-58.52502765,-34.63601265,AUTO-AUTO,AUTO,AUTO,-1.0
599,2021-0001,1,2021-01-06,2021,1,6,18:00:00,18,AU FRONDIZI KM. 3,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AU FRONDIZI KM. 3,1,-58.37960727,-34.62622851,MOTO-AUTO,MOTO,AUTO,-1.0
601,2021-0003,1,2021-01-10,2021,1,10,07:40:00,7,"AU 25 DE MAYO KM. 3,7",AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO KM. 3.7,7,-58.43052224,-34.62941648,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,-1.0
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,NaN,NaN,MOTO-PASAJEROS,MOTO,PASAJEROS,-1.0
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0
631,2021-0033,1,2021-04-14,2021,4,14,17:50:00,17,AU 25 DE MAYO ALTURA BAJADA BOEDO AV.,AUTOPISTA,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO y BOEDO AV.,5,-58.41595919,-34.62743346,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO,-1.0
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,"AU BUENOS AIRES LA PLATA KM 4,5",4,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0


- Con la información anteorio podemos inferir que si el tipo de calle es una autopista, no es posible que sea un cruce, por lo que remplazaremos estos datos con 0.

In [174]:
hechos.loc[(hechos['Cruce_boolean']==-1) & (hechos['tipo_de_calle'] == 'AUTOPISTA'),'Cruce_boolean'] = 0 

Observando el lugar_del_hecho, notamos que existen dos registros que tiene la información de la altura, por lo que procederemos a remplazar -1 por 0. a las victimas con ids 2017-0042 y 2020-0039

In [175]:
hechos[hechos['Cruce_boolean']==-1]

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,-1.0
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,MURGUIONDO,9,NaN,NaN,PEATON-CARGAS,PEATON,CARGAS,-1.0


In [176]:
hechos.loc[(hechos['Cruce_boolean']==-1) & hechos['id_hechos'].isin(['2017-0042','2020-0039']), 'Cruce_boolean'] = 0 

In [153]:
hechos.loc[hechos['id_hechos'].isin(['2017-0042','2020-0039'])] 

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",14,NaN,NaN,MOTO-CARGAS,MOTO,CARGAS,0.0
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,MURGUIONDO,9,NaN,NaN,PEATON-CARGAS,PEATON,CARGAS,0.0


In [177]:
hechos[hechos['Cruce_boolean']==-1]

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,SD,0,NaN,NaN,PEATON-SD,PEATON,SD,-1.0


Dado que solo tenemos un dato faltante y la victima es un peaton , remplazaremos con el dato más repetido en Cruce_boolean, ya que observamos que la mayoria de los homicidios de peatones ocurre en los cruces.

In [189]:
pd.DataFrame(hechos[hechos['victimas'] =='PEATON']['Cruce_boolean'].value_counts())

,count
Cruce_boolean,
1.0,207
0.0,56
-1.0,1


In [193]:
hechos.loc[hechos['id_hechos']=='2016-0151','Cruce_boolean'] = 1

In [194]:
hechos

,id_hechos,n_victimas,fecha,anho,mes,dia,hora,franja_horaria,lugar_del_hecho,tipo_de_calle,calle,direccion_normalizada,comuna,longitud,latitud,participantes,victimas,acusado,Cruce_boolean
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,1.0
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,1.0
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,0.0
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,1.0
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,RIESTRA AV.,RIESTRA AV. y MOM,7,-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO,1.0
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.","DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO,1.0
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,GAONA AV.,GAONA AV. y TERRADA,11,-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS,1.0
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.","PERON, EVA AV. 4071",9,-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS,0.0


### Carga Hechos

In [195]:
path_hechos_clear = os.path.join('..','data','clear','hechos_homicidios.csv')

In [196]:
hechos.to_csv(path_hechos_clear,index=False)

# ETL Victimas

#### Extracción Victimas

In [199]:
path_homicidios = os.path.join('..','data','raw','homicidios.xlsx')

In [200]:
victimas = pd.read_excel(path_homicidios, sheet_name='VICTIMAS')

In [208]:
nombres_columnas_victimas = {
    'ID_hecho': 'id_hechos',
    'FECHA': 'fecha',
    'AAAA': 'anho',
    'MM': 'mes',
    'DD': 'dia',
    'ROL': 'rol',
    'VICTIMA': 'victimas',
    'SEXO': 'sexo',
    'EDAD': 'edad',
    'FECHA_FALLECIMIENTO': 'fecha_fallecimiento'
}

victimas.rename(columns=nombres_columnas_victimas, inplace=True)

In [211]:
victimas.head(5)

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [212]:
victimas.columns

Index(['id_hechos', 'fecha', 'anho', 'mes', 'dia', 'rol', 'victimas', 'sexo',
       'edad', 'fecha_fallecimiento'],
      dtype='object')

#### Transformación Victimas

Transformaremos la columna fecha de falleciento para descartar la hora, ya que ningun registro cuenta la hora del homicidio.

In [239]:
victimas["fecha_fallecimiento"] = pd.to_datetime(victimas["fecha_fallecimiento"], format='%Y-%m-%d', errors='coerce')

In [241]:
victimas.dtypes

id_hechos                      object
fecha                  datetime64[ns]
anho                            int64
mes                             int64
dia                             int64
rol                            object
victimas                       object
sexo                           object
edad                           object
fecha_fallecimiento    datetime64[ns]
dtype: object

In [236]:
victimas

,id_hechos,fecha,anho,mes,dia,rol,victimas,sexo,edad,fecha_fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00


### Exportar Victimas

In [460]:
path_victimas_clear = os.path.join('..','data','clear','victimas_homicidios.csv')

victimas.to_csv(path_victimas_clear, index=False)


# ETL Lesiones

##### Extracción Lesiones

#### Transformación Lesiones

#### Exportar Lesiones